## Model-to-model comparison

You can also use Brain-Score to compare how similar models are to one another.

### Behavioral comparison

Let's compare the reading times predictions of two models:

In [1]:
from brainscore_language import load_model, ArtificialSubject, load_metric

# load models
model1 = load_model('distilgpt2')
model2 = load_model('gpt2-xl')

# perform task
model1.perform_behavioral_task(ArtificialSubject.Task.reading_times)
model2.perform_behavioral_task(ArtificialSubject.Task.reading_times)
text = ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
reading_times1 = model1.digest_text(text)
reading_times2 = model2.digest_text(text)

# compare
metric = load_metric('pearsonr')
score = metric(reading_times1, reading_times2)
print(score)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

### Neural comparison

Similarly, you can compare how similar neural activity is in two models.
Here, we compare two artificial subject models stemming from the same base model by choosing different layers, but you can also compare different models altogether like above.

In [ ]:
from brainscore_language import ArtificialSubject, load_metric
from brainscore_language.model_helpers.huggingface import HuggingfaceSubject

# load models
model1 = HuggingfaceSubject(model_id='distilgpt2', region_layer_mapping={
        ArtificialSubject.RecordingTarget.language_system: 'transformer.h.4.ln_1'})
model2 = HuggingfaceSubject(model_id='distilgpt2', region_layer_mapping={
        ArtificialSubject.RecordingTarget.language_system: 'transformer.h.5.ln_1'})

# record neural activity
model1.perform_neural_recording(recording_target=ArtificialSubject.RecordingTarget.language_system,
                                recording_type=ArtificialSubject.RecordingType.fMRI)
model2.perform_neural_recording(recording_target=ArtificialSubject.RecordingTarget.language_system,
                                recording_type=ArtificialSubject.RecordingType.fMRI)
text = ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
activity1 = model1.digest_text(text)
activity2 = model2.digest_text(text)

# compare
metric = load_metric('linear_pearsonr')
score = metric(activity1, activity2)
print(score)

## Human-to-human comparison

As with the model-to-model comparisons, you can compare humans to one another.
In this case, the data has already been recorded so we simply compare two sets of data to one another.


### Behavioral comparison

Using data from Futrell et al. 2018, we can compare how similar the reading times of half the subjects are to the reading times of the other half of subjects (this is also part of how the ceiling is estimated in the [Futrell2018 reading times benchmark](https://github.com/brain-score/language/blob/main/brainscore_language/benchmarks/futrell2018/__init__.py)):

In [ ]:
from numpy.random import RandomState
from brainscore_language import load_dataset, load_metric

# load data
data = load_dataset('Futrell2018')
print(data)

In [ ]:
# split into halves
random = RandomState(0)
subjects = data['subject_id'].values
half1_subjects = random.choice(subjects, size=len(subjects) // 2, replace=False)
half2_subjects = set(subjects) - set(half1_subjects)
half1 = data.sel(subject_id=half1_subjects)
half2 = data.sel(subject_id=half2_subjects) # todo: or data[{'subject': [subject_id in half1_subjects for subject_id in subjects]}]

# compare
metric = load_metric('pearsonr')
score = metric(half1, half2)